# NS_03 Group Project

## Part1

In [6]:
import torch
import os
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import datetime
import sklearn.metrics

In [7]:
# load data_set, and calculate the mean and std of dataset

# transform = transforms.Compose([
#     transforms.Resize((256, 256)),  # 缩放到224 * 224
#     transforms.ToTensor()
# ])

# # 0 -> mask  1-> nonmask  2 -> not a person
# train_dataset = ImageFolder('./data/train', transform=transform)

# # calculate mean of imgs in each RGB channel
# imgs = torch.stack([img_t for img_t, _ in train_dataset], dim=3)
# print(imgs.shape)

# imgs.view(3, -1).mean(dim=1)  # mean of (number of imgs) in each channel
# imgs.view(3, -1).std(dim=1) # std of (number of imgs) in each channel

In [8]:
# load dataset, normarlize it
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 缩放到224 * 224
    # transforms.CenterCrop(256)   #中心剪裁后四周padding补充 (后续可以padding)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4990, 0.4567, 0.4188], std=[0.2913, 0.2778, 0.2836]) 
])

# 0 -> mask  1-> nonmask  2 -> not a person
train_dataset = ImageFolder('./data/train', transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = ImageFolder('./data/test', transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [9]:
# cnn model

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1), # in=3x256x256; out=32x256x256
            nn.ReLU(),
            nn.MaxPool2d(2), # out=32x128x128
            nn.Conv2d(16, 16, kernel_size=3, padding=1), # in=32x128x128, out=16x128x128
            nn.ReLU(),
            nn.MaxPool2d(2), # out=16x64x64
            nn.Conv2d(16, 32, kernel_size=3, padding=1), # in=16x64x64, out=8x64x64
            nn.ReLU(),
            nn.MaxPool2d(2), # out=8x32x32,
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2) # 16x16
        )
        
        self.fc_layer = nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(64*16*16, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(1024, 32),
            nn.ReLU(),
            nn.Dropout(p=0.1),
            nn.Linear(32, 3)
        )
    
    def forward(self, x):
        # conv layer
        x = self.conv_layer(x)
        
        # flatten
        x = x.view(x.size(0), -1)
        
        # fc layer
        x = self.fc_layer(x)
        
        return x


In [10]:
device = (torch.device('cuda') if torch.cuda.is_available() 
         else torch.device('cpu'))
print(f"Training on device {device}.")

Training on device cpu.


In [11]:
# define train_loop function
def train_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        for imgs, labels in train_dataloader: # loop over batches in dataset
            # move data to GPU if available
            imgs = imgs.to(device=device)  
            labels = labels.to(device=device)
            
            outputs = model(imgs)  # feed a batch through our model
            
            loss = loss_fn(outputs, labels)  # computes the loss
            
            optimizer.zero_grad()  # getting rid of the gradients from the last round
            
            loss.backward()  # performs backward step, compute the gradients of all parameters
            
            optimizer.step()  # updates the model
            
            loss_train += loss.item() # sums of losses we saw over the epoch
            
        # print the average loss per batch, in epoch%10 == 0 
        print('{} Epoch {}, Training loss {}'.format(
            datetime.datetime.now(), epoch, loss_train/len(train_loader)
        ))

In [12]:
model = CNN().to(device=device)  # instantiates cnn model

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list   # number of parameters, and their shapes

(16837043,
 [432, 16, 2304, 16, 4608, 32, 18432, 64, 16777216, 1024, 32768, 32, 96, 3])

In [13]:
os.listdir()

['use_model.ipynb',
 '.DS_Store',
 'use_model.py',
 'cnn.ipynb',
 'my_model_acc83.pkl',
 'NS_03_evaluation.py',
 'README.md',
 'train_and_test.py',
 'MaskDetection.ipynb',
 'NS_03_part01.py',
 'my_model.pkl',
 '.ipynb_checkpoints',
 '.git',
 'data']

In [14]:
# perform training

learning_rate = 0.001
# optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()  # use cross entropy loss function

if 'my_model_acc83.pkl' not in os.listdir():
    # call train_loop() function
    train_loop(
        n_epochs = 15,
        optimizer = optimizer,
        model = model,
        loss_fn = loss_fn,
        train_loader = train_dataloader
    )
    torch.save(model, 'my_model2.pkl')
else:
    model = torch.load('my_model_acc83.pkl')

In [15]:
# define validate function
def validate(model, loaders, names):
    model.eval()
    # accuracy on training data and test data
    for name, loader in zip(names, loaders):
        correct = 0
        total = 0
        total_predicted = []
        total_labels = []
        
        with torch.no_grad(): # do not want gradients here, as we will not want to update parameters
            for imgs, labels in loader:
                # move data to GPU if available
                imgs = imgs.to(device=device)  
                labels = labels.to(device=device)
                total_labels.append(labels)
                
                # feed input to models
                outputs = model(imgs)  
                
                # gives the index of the highest value as output
                _, predicted = torch.max(outputs, dim=1)  
                total_predicted.append(predicted)
                
                # counts the number of example, total is increased by the batch size
                total += labels.shape[0]  
            
                # the prediction and ground truth agree
                correct += int((predicted == labels).sum()) 
                
        total_predicted = torch.hstack(total_predicted).cpu()
        total_labels = torch.hstack(total_labels).cpu()
        
        print("Accuracy {}: {:.2f}".format(name, correct / total))  
        print(sklearn.metrics.classification_report(total_labels, total_predicted))
        print(sklearn.metrics.confusion_matrix(total_labels, total_predicted))

In [16]:
# measuring accuracy
validate(model, [train_dataloader, test_dataloader], ['train', 'test'])

Accuracy train: 0.95
              precision    recall  f1-score   support

           0       0.98      0.93      0.95       284
           1       0.94      0.94      0.94       338
           2       0.94      0.96      0.95       576

    accuracy                           0.95      1198
   macro avg       0.95      0.94      0.95      1198
weighted avg       0.95      0.95      0.95      1198

[[264   5  15]
 [  0 318  20]
 [  6  16 554]]
Accuracy test: 0.83
              precision    recall  f1-score   support

           0       0.89      0.68      0.77       123
           1       0.80      0.98      0.88       124
           2       0.83      0.83      0.83       102

    accuracy                           0.83       349
   macro avg       0.84      0.83      0.83       349
weighted avg       0.84      0.83      0.83       349

[[ 84  23  16]
 [  1 122   1]
 [  9   8  85]]


## Part 2

- Run model on Female test dataset and Male dataset, evaluate their performances.
- Compare their performances, check whether our model has gender bias.

In [17]:
model = torch.load('my_model_acc83.pkl')
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))

male_dataset = ImageFolder('./data/test_male', transform=transform)
male_dataloader = DataLoader(male_dataset, batch_size=32, shuffle=True)


female_dataset = ImageFolder('./data/test_female', transform=transform)
female_dataloader = DataLoader(female_dataset, batch_size=32, shuffle=True)

validate(model, [male_dataloader, female_dataloader], ['Male', 'Female'])


Accuracy Male: 0.79
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        93
           1       0.81      0.87      0.84       105
           2       0.00      0.00      0.00         0

    accuracy                           0.79       198
   macro avg       0.60      0.52      0.55       198
weighted avg       0.90      0.79      0.83       198

[[65 21  7]
 [ 0 91 14]
 [ 0  0  0]]
/Users/zoufan/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Accuracy Female: 0.78
              precision    recall  f1-score   support

           0       0.98      0.65      0.78        91
           1       0.85      0.88      0.86       109
           2       0.00      0.00      0.00         0

  